In [85]:
import os
import sys
import gzip
import code
import urllib
import numpy as np
from PIL import Image
import tensorflow as tf
import tensorflow.python.platform
import matplotlib.image as mpimg
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split

In [139]:
%reload_ext autoreload
%autoreload 2

In [180]:
sys.path.append('./')
from helpers.helpers import *
from helpers.mask_to_submission import *

In [105]:
NUM_CHANNELS = 3  # RGB images
PIXEL_DEPTH = 255
NUM_LABELS = 2
TRAINING_SIZE = 80
VALIDATION_SIZE = 5  # Size of the validation set.
SEED = 66478  # Set to None for random seed.
BATCH_SIZE = 16  # 64
NUM_EPOCHS = 10
RESTORE_MODEL = False  # If True, restore existing model instead of training a new one
RECORDING_STEP = 0

# Set image patch size in pixels
# IMG_PATCH_SIZE should be a multiple of 4
# image size should be an integer multiple of this number!
IMG_PATCH_SIZE = 16

In [89]:
data_dir = 'data/training/'
train_data_filename = data_dir + 'images/'
train_labels_filename = data_dir + 'groundtruth/' 

# Extract patches into numpy arrays. Dim: (nb patch, 16, 16, 3)
training_data = extract_data(train_data_filename, 100)
training_labels = extract_labels(train_labels_filename, 100)

Loaded 100 training images
Loaded 100 groudtruth images


In [100]:
X_train, X_validation, y_train, y_validation = train_test_split(training_data, training_labels,\
                                                    train_size= TRAINING_SIZE/100, random_state=SEED)

print(X_train.shape)
print(X_validation.shape)

(50000, 16, 16, 3)
(12500, 16, 16, 3)


In [90]:
num_epochs = NUM_EPOCHS

c0 = 0  # bgrd
c1 = 0  # road
for i in range(len(y_train)):
    if y_train[i][0] == 1:
        c0 = c0 + 1
    else:
        c1 = c1 + 1
print('Number of data points per class: c0 = ' + str(c0) + ' c1 = ' + str(c1))

print('Balancing training data...')
min_c = min(c0, c1)
idx0 = [i for i, j in enumerate(y_train) if j[0] == 1]
idx1 = [i for i, j in enumerate(y_train) if j[1] == 1]
new_indices = idx0[0:min_c] + idx1[0:min_c]
print(len(new_indices))
print(X_train.shape)
X_train = X_train[new_indices, :, :, :]
y_train = y_train[new_indices]

train_size = y_train.shape[0]

c0 = 0
c1 = 0
for i in range(len(y_train)):
    if y_train[i][0] == 1:
        c0 = c0 + 1
    else:
        c1 = c1 + 1
print('Number of data points per class: c0 = ' + str(c0) + ' c1 = ' + str(c1))

Number of data points per class: c0 = 37053 c1 = 12947
Balancing training data...
25894
(50000, 16, 16, 3)
Number of data points per class: c0 = 12947 c1 = 12947


In [91]:
X_train.shape

(25894, 16, 16, 3)

In [92]:
if RESTORE_MODEL:
    # It can be used to reconstruct the model identically.
    model = models.load_model("handmade_cnn_save/")

In [102]:
if not RESTORE_MODEL:
    model = models.Sequential()
    model.add(layers.Conv2D(32, (5, 5), activation='relu', input_shape=(16, 16, 3), padding='same', use_bias=True))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(64, (5, 5), activation='relu', input_shape=(8, 8, 3), padding='same', use_bias=True))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Flatten())
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(2))

In [103]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 16, 16, 32)        2432      
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 8, 8, 32)          0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 8, 8, 64)          51264     
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 4, 4, 64)          0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 1024)              0         
_________________________________________________________________
dense_6 (Dense)              (None, 64)                65600     
_________________________________________________________________
dense_7 (Dense)              (None, 2)                

In [104]:
if not RESTORE_MODEL: 
    model.compile(optimizer='adam',
              loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
    history = model.fit(X_train, y_train, epochs=NUM_EPOCHS ,validation_data=(X_validation, y_validation))

Train on 50000 samples, validate on 12500 samples
Epoch 1/100
50000/50000 [==============================] - 31s 614us/sample - loss: 0.5197 - accuracy: 0.7436 - val_loss: 0.5145 - val_accuracy: 0.7270
Epoch 2/100
50000/50000 [==============================] - 30s 607us/sample - loss: 0.4719 - accuracy: 0.7627 - val_loss: 0.4569 - val_accuracy: 0.7655
Epoch 3/100
50000/50000 [==============================] - 32s 635us/sample - loss: 0.4310 - accuracy: 0.7860 - val_loss: 0.4522 - val_accuracy: 0.7769
Epoch 4/100
50000/50000 [==============================] - 24s 486us/sample - loss: 0.4132 - accuracy: 0.7984 - val_loss: 0.4012 - val_accuracy: 0.8088
Epoch 5/100
50000/50000 [==============================] - 24s 476us/sample - loss: 0.3748 - accuracy: 0.8209 - val_loss: 0.3936 - val_accuracy: 0.8158
Epoch 6/100
50000/50000 [==============================] - 28s 554us/sample - loss: 0.3510 - accuracy: 0.8348 - val_loss: 0.3936 - val_accuracy: 0.8141
Epoch 7/100
50000/50000 [=============

KeyboardInterrupt: 

In [145]:
if not RESTORE_MODEL:
    model.save("handmade_cnn_save")

INFO:tensorflow:Assets written to: handmade_cnn_save/assets


In [132]:
pred_train = model.predict(X_train)
print("Training error rate: {:.2f}%".format(error_rate(pred_train, y_train)))

Training error rate: 9.66%


In [125]:
pred_validation = model.predict(X_validation)
print("Validation error rate: {:.2f}%".format(error_rate(pred_validation, y_validation)))

Validation error rate: 17.78%


In [144]:
F1_score(y_train, pred_train)

0.7976718867766519

In [146]:
print("Running prediction on training set")
prediction_training_dir = "predictions_training/"
if not os.path.isdir(prediction_training_dir):
    os.mkdir(prediction_training_dir)
for i in range(1, TRAINING_SIZE + 1):
    pimg = get_prediction_with_groundtruth(model, train_data_filename, i)
    Image.fromarray(pimg).save(prediction_training_dir + "prediction_" + str(i) + ".png")
    oimg = get_prediction_with_overlay(model, train_data_filename, i)
    oimg.save(prediction_training_dir + "overlay_" + str(i) + ".png")  

Running prediction on training set


In [179]:
predict_test_masks(model)

Running prediction on test set


In [186]:
masks_to_submission("submission.csv", "predictions_test/")

predictions_test/test_14.png
predictions_test/test_28.png
predictions_test/test_29.png
predictions_test/test_15.png
predictions_test/test_9.png
predictions_test/test_17.png
predictions_test/test_16.png
predictions_test/test_8.png
predictions_test/test_12.png
predictions_test/test_13.png
predictions_test/test_39.png
predictions_test/test_11.png
predictions_test/test_10.png
predictions_test/test_38.png
predictions_test/test_48.png
predictions_test/test_49.png
predictions_test/test_42.png
predictions_test/test_43.png
predictions_test/test_41.png
predictions_test/test_40.png
predictions_test/test_44.png
predictions_test/test_50.png
predictions_test/test_45.png
predictions_test/test_47.png
predictions_test/test_46.png
predictions_test/test_3.png
predictions_test/test_35.png
predictions_test/test_21.png
predictions_test/test_20.png
predictions_test/test_34.png
predictions_test/test_2.png
predictions_test/test_22.png
predictions_test/test_36.png
predictions_test/test_37.png
predictions_test/t